# Imports

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA

from sklearn.datasets import *

from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import copy

from data_structures.tree_classifier import TreeClassifier
import utils.utils
import time

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Prepare Data

In [ ]:
# Download all datasets from sklearn
for m in [fetch_olivetti_faces, fetch_20newsgroups_vectorized, fetch_lfw_people, fetch_lfw_pairs, fetch_covtype, fetch_rcv1, fetch_kddcup99, fetch_california_housing]:
    print(m)
    try:
        all_ = m()
        train = m(subset='train')
        test = m(subset='test')
    except:
        pass

In [2]:
# Download the data from two categories
cats = ['alt.atheism', 'sci.space']
ng_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'), categories=cats)
ng_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'), categories=cats)


vectorizer = TfidfVectorizer()
trans = vectorizer.fit(ng_train.data)
train_vectors = vectorizer.transform(ng_train.data)
test_vectors = vectorizer.transform(ng_test.data)
print("Number of datapoints: ", len(ng_train.data))
print("Number of features: ", train_vectors.shape[1])
print("Balance: ", np.sum(ng_train.target) / len(ng_train.target)) # 55-45, roughly balanced

N_COMPONENTS=100
pca = PCA(n_components=N_COMPONENTS)
pca.fit(train_vectors.toarray())
pca_train_vecs = pca.transform(train_vectors.toarray())
pca_test_vecs = pca.transform(test_vectors.toarray())

classes_arr = np.unique(ng_train.target)
classes = utils.utils.class_to_idx(classes_arr)

Number of datapoints:  1073
Number of features:  18217
Balance:  0.5526561043802423


# Compare our implementation's accuracy to sklearn

In [3]:
dt = DecisionTreeClassifier(random_state=0)
dt.fit(pca_train_vecs,ng_train.target)
print("sklearn Decision Tree Accuracy:", np.mean(dt.predict(pca_test_vecs) == ng_test.target))

#cross_val_score(dt, pca_train_vecs, ng_train.target, cv=10).mean()

sklearn Decision Tree Accuracy: 0.782608695652174


In [4]:
rf = RandomForestClassifier(random_state=0)
rf.fit(pca_train_vecs,ng_train.target)
print("sklearn Random Forest Accuracy:", np.mean(rf.predict(pca_test_vecs) == ng_test.target))

#cross_val_score(rf, pca_train_vecs, ng_train.target, cv=10).mean()

sklearn Random Forest Accuracy: 0.8050490883590463


In [5]:
tc = TreeClassifier(data=pca_train_vecs, labels=ng_train.target, max_depth=5, classes=classes, verbose=False)
start = time.time()
tc.fit()
end = time.time()
print("Train accuracy:", np.mean(tc.predict_batch(pca_train_vecs)[0] == ng_train.target))
print("Test accuracy:", np.mean(tc.predict_batch(pca_test_vecs)[0] == ng_test.target))
print("Num queries:", tc.num_queries)
print("Runtime:", end-start)

Train accuracy: 0.8704566635601119
Test accuracy: 0.7868162692847125
Num queries: 4406
Runtime: 6.60705304145813


In [6]:
tc = TreeClassifier(data=pca_train_vecs, labels=ng_train.target, max_depth=5, classes=classes, solver="EXACT", verbose=False)
start = time.time()
tc.fit()
end = time.time()
print("Train accuracy:", np.mean(tc.predict_batch(pca_train_vecs)[0] == ng_train.target))
print("Test accuracy:", np.mean(tc.predict_batch(pca_test_vecs)[0] == ng_test.target))
print("Num queries:", tc.num_queries)
print("Runtime:", end-start)

Train accuracy: 0.8704566635601119
Test accuracy: 0.7868162692847125
Num queries: 5300
Runtime: 5.1996259689331055


# Make the dataset huge

In [7]:
doublings = 4
pca_train_vecs_huge = copy.deepcopy(pca_train_vecs)
pca_train_labels_huge = copy.deepcopy(ng_train.target)
print(pca_train_vecs_huge.shape)
for i in range(doublings):
    pca_train_vecs_huge = np.concatenate((pca_train_vecs_huge, pca_train_vecs_huge))
    pca_train_labels_huge = np.concatenate((pca_train_labels_huge, pca_train_labels_huge))
print(pca_train_vecs_huge.shape)

(1073, 100)
(17168, 100)


In [8]:
tc = TreeClassifier(data=pca_train_vecs_huge, labels=pca_train_labels_huge, max_depth=2, classes=classes, verbose=True, random_state=0)
start = time.time()
tc.fit()
end = time.time()
print("Train accuracy:", np.mean(tc.predict_batch(pca_train_vecs)[0] == ng_train.target))
print("Test accuracy:", np.mean(tc.predict_batch(pca_test_vecs)[0] == ng_test.target))
print("Num queries:", tc.num_queries)
print("Runtime:", end-start)
tc.tree_print()

Calculated split with 100 queries
Calculated split with 5400 queries
Calculated split with 10900 queries
Fitting finished
Train accuracy: 0.8070829450139795
Test accuracy: 0.761570827489481
Num queries: 16400
Runtime: 6.1677350997924805
|--- feature_1 <= -0.037112255696725716
|   |--- feature_3 <= 0.11135085961226182
|   |   |--- class: 0
|   |--- feature_3 > 0.11135085961226182
|   |   |--- class: 1
|--- feature_1 > -0.037112255696725716
|   |--- feature_3 <= -0.07696973751446881
|   |   |--- class: 0
|   |--- feature_3 > -0.07696973751446881
|   |   |--- class: 1




In [9]:
# We should implement vanilla TreeClassifier --> which uses identity bins

tc = TreeClassifier(data=pca_train_vecs_huge, labels=pca_train_labels_huge, max_depth=2, classes=classes, solver="EXACT", verbose=True, random_state=0)
start = time.time()
tc.fit()
end = time.time()
print("Train accuracy:", np.mean(tc.predict_batch(pca_train_vecs)[0] == ng_train.target))
print("Test accuracy:", np.mean(tc.predict_batch(pca_test_vecs)[0] == ng_test.target))
print("Num queries:", tc.num_queries)
print("Runtime:", end-start)
tc.tree_print()

Calculated split with 17168 queries
Calculated split with 5728 queries
Calculated split with 11440 queries
Fitting finished
Train accuracy: 0.8070829450139795
Test accuracy: 0.761570827489481
Num queries: 34336
Runtime: 22.784740209579468
|--- feature_1 <= -0.037112255696725716
|   |--- feature_3 <= 0.11135085961226182
|   |   |--- class: 0
|   |--- feature_3 > 0.11135085961226182
|   |   |--- class: 1
|--- feature_1 > -0.037112255696725716
|   |--- feature_3 <= -0.07696973751446881
|   |   |--- class: 0
|   |--- feature_3 > -0.07696973751446881
|   |   |--- class: 1




# Verify our implementation of baseline models agrees with sklearn's in terms of accuracy

# Classification:

In [16]:
from data_structures.wrappers.random_forest_classifier import RandomForestClassifier as RFC_ours
from data_structures.wrappers.extremely_random_forest_classifier import ExtremelyRandomForestClassifier as ERFC_ours

from data_structures.wrappers.random_forest_regressor import RandomForestRegressor as RFR_ours
from data_structures.wrappers.extremely_random_forest_regressor import ExtremelyRandomForestRegressor as ERFR_ours

from sklearn.ensemble import RandomForestClassifier as RFC_sklearn
from sklearn.ensemble import ExtraTreesClassifier as ERFC_sklearn

from sklearn.ensemble import RandomForestRegressor as RFR_sklearn
from sklearn.ensemble import ExtraTreesRegressor as ERFR_sklearn

from utils.constants import GINI, BEST, EXACT, MSE

# TODO(@motiwari): Allow for gradient boosted comparisons as well
def compare_accuracies(
    compare: str = "RFC",
    train_data: np.ndarray = None,
    train_targets: np.ndarray = None,
    test_data: np.ndarray = None,
    test_targets: np.ndarray = None,
    num_seeds: int = 10,
) -> bool:
    our_train_accs = []
    our_test_accs = []
    their_train_accs = []
    their_test_accs = []
    for seed in range(num_seeds):
        # Ok to have n_jobs = -1 throughout?
        if compare == "RFC":
            our_model = RFC_ours(data=train_data, labels=train_targets, n_estimators=5, max_depth=5, min_samples_split=2, min_impurity_decrease=0, max_leaf_nodes=None, budget=None, criterion=GINI, splitter=BEST, solver=EXACT, random_state=seed, verbose=False)
            their_model = RFC_sklearn(n_estimators=5, criterion='gini', max_depth=5, min_samples_split=2, min_impurity_decrease=0, max_leaf_nodes=None, n_jobs=-1, random_state=seed, verbose=0)
        elif compare == "ERFC":
            our_model = ERFC_ours(data=train_data, labels=train_targets, n_estimators=5, max_depth=5, num_bins=None, min_samples_split=2, min_impurity_decrease=0, max_leaf_nodes=None, budget=None, criterion=GINI, splitter=BEST, solver=EXACT, random_state=seed, verbose=False)
            their_model = ERFC_sklearn(n_estimators=5, criterion='gini', max_depth=5, min_samples_split=2, max_features='auto', max_leaf_nodes=None, min_impurity_decrease=0.0, bootstrap=False, n_jobs=-1, random_state=seed, verbose=0)
        elif compare == "RFR":
            our_model = RFR_ours(data=train_data, labels=train_targets, n_estimators=1, max_depth=5, min_samples_split=2, min_impurity_decrease=0, max_leaf_nodes=None, budget=None, criterion=MSE, splitter=BEST, solver=EXACT, random_state=seed, verbose=False)
            their_model = RFR_sklearn(n_estimators=1, criterion='squared_error', max_depth=5, min_samples_split=2, max_leaf_nodes=None, min_impurity_decrease=0.0, bootstrap=True, n_jobs=-1, random_state=seed, verbose=0)    
        elif compare == "ERFR":
            our_model = ERFR_ours(data=train_data, labels=train_targets, n_estimators=5, max_depth=5, num_bins=None, min_samples_split=2, min_impurity_decrease=0, max_leaf_nodes=None, budget=None, criterion=MSE, splitter=BEST, solver=EXACT, random_state=seed, verbose=False)
            their_model = ERFR_sklearn(n_estimators=5, criterion='squared_error', max_depth=5, min_samples_split=2, max_features='auto', min_impurity_decrease=0.0, bootstrap=False, n_jobs=-1, random_state=seed, verbose=0)
        else:
            raise NotImplementedError("Need to decide what models to compare")
        
        our_model.fit()
        their_model.fit(train_data, train_targets)

        if compare == "RFC" or compare == "ERFC" or compare == "HRFC":
            our_train_acc = np.mean(our_model.predict_batch(train_data)[0] == train_targets)
            our_test_acc = np.mean(our_model.predict_batch(test_data)[0] == test_targets)
            their_train_acc = np.mean(their_model.predict(train_data) == train_targets)
            their_test_acc = np.mean(their_model.predict(test_data) == test_targets)
        elif compare == "RFR" or compare == "ERFR" or compare == "HRFR":
            our_train_acc = np.mean((our_model.predict_batch(train_data) - train_targets)**2)
            our_test_acc = np.mean((our_model.predict_batch(test_data) - test_targets)**2)
            their_train_acc = np.mean((our_model.predict_batch(train_data) - train_targets)**2)
            their_test_acc = np.mean((our_model.predict_batch(test_data) - test_targets)**2)

        print("(Ours) Train accuracy:", our_train_acc)
        print("(Ours) Test accuracy:", our_test_acc)
        print("(Theirs) Train accuracy:", their_train_acc)
        print("(Theirs) Test accuracy:", their_test_acc)
        print("-" * 30)

        our_train_accs.append(our_train_acc)
        our_test_accs.append(our_test_acc)
        their_train_accs.append(their_train_acc)
        their_test_accs.append(their_test_acc)
    
    our_avg_train = np.mean(our_train_accs)
    our_std_train = np.std(our_train_accs)

    our_avg_test = np.mean(our_test_accs)
    our_std_test = np.std(our_test_accs)
    
    their_avg_train = np.mean(their_train_accs)
    their_std_train = np.std(their_train_accs)
    
    their_avg_test = np.mean(their_test_accs)
    their_std_test = np.std(their_test_accs)
    
    # See if confidence intervals overlap
    overlap = np.abs(their_avg_test - our_avg_test) < their_std_test + our_std_test
    return overlap, our_avg_train, our_std_train, our_avg_test, our_std_test, their_avg_train, their_std_train, their_avg_test, their_std_test

In [11]:
train_data = pca_train_vecs
train_labels = ng_train.target
test_data = pca_test_vecs
test_labels = ng_test.target

compare_accuracies("RFC", train_data, train_labels, test_data, test_labels)

(Ours) Train accuracy: 0.869524697110904
(Ours) Test accuracy: 0.7812061711079944
(Theirs) Train accuracy: 0.8741845293569431
(Theirs) Test accuracy: 0.7643758765778401
------------------------------
(Ours) Train accuracy: 0.8574091332712023
(Ours) Test accuracy: 0.7601683029453016
(Theirs) Train accuracy: 0.8779123951537745
(Theirs) Test accuracy: 0.7713884992987378
------------------------------
(Ours) Train accuracy: 0.8797763280521901
(Ours) Test accuracy: 0.7784011220196353
(Theirs) Train accuracy: 0.8872320596458527
(Theirs) Test accuracy: 0.7601683029453016
------------------------------
(Ours) Train accuracy: 0.8797763280521901
(Ours) Test accuracy: 0.7643758765778401
(Theirs) Train accuracy: 0.8704566635601119
(Theirs) Test accuracy: 0.7812061711079944
------------------------------
(Ours) Train accuracy: 0.8527493010251631
(Ours) Test accuracy: 0.7657784011220197
(Theirs) Train accuracy: 0.8583410997204101
(Theirs) Test accuracy: 0.7727910238429172
---------------------------

(True,
 0.8671015843429636,
 0.018658890015434363,
 0.7479663394109396,
 0.03927594635191804,
 0.8681267474370922,
 0.018415477811157405,
 0.7593267882187938,
 0.027161219493767656)

In [12]:
compare_accuracies("ERFC", train_data, train_labels, test_data, test_labels)

(Ours) Train accuracy: 0.7297297297297297
(Ours) Test accuracy: 0.6479663394109397
(Theirs) Train accuracy: 0.8136067101584343
(Theirs) Test accuracy: 0.7475455820476858
------------------------------
(Ours) Train accuracy: 0.7427772600186393
(Ours) Test accuracy: 0.6339410939691444
(Theirs) Train accuracy: 0.7176141658900279
(Theirs) Test accuracy: 0.6283309957924264
------------------------------
(Ours) Train accuracy: 0.6877912395153775
(Ours) Test accuracy: 0.5960729312762973
(Theirs) Train accuracy: 0.7269338303821062
(Theirs) Test accuracy: 0.6100981767180925
------------------------------
(Ours) Train accuracy: 0.7110904007455732
(Ours) Test accuracy: 0.6577840112201964
(Theirs) Train accuracy: 0.7399813606710158
(Theirs) Test accuracy: 0.6311360448807855
------------------------------
(Ours) Train accuracy: 0.782851817334576
(Ours) Test accuracy: 0.7138849929873773
(Theirs) Train accuracy: 0.7260018639328985
(Theirs) Test accuracy: 0.6283309957924264
---------------------------

(True,
 0.7247903075489284,
 0.035671434343217547,
 0.6389901823281907,
 0.04445892194481986,
 0.7387698042870456,
 0.03184285378036402,
 0.6479663394109397,
 0.04105350098507873)

# Regression: 

### Weirdly, we get exactly the same results as sklearn for all seeds. We should investigate whether this is a bug

In [13]:
np.random.seed(0)

data, targets = fetch_california_housing(return_X_y=True)
random_idcs = np.random.choice(len(data), size=len(data),replace=False)
data = data[random_idcs]
targets = targets[random_idcs]

TRAIN_TEST_SPLIT = 16000
train_data = data[:TRAIN_TEST_SPLIT]
train_targets = targets[:TRAIN_TEST_SPLIT]

test_data = data[TRAIN_TEST_SPLIT:]
test_targets = targets[TRAIN_TEST_SPLIT:]

In [17]:
compare_accuracies("RFR", train_data, train_targets, test_data, test_targets)

(Ours) Train accuracy: 0.6434856262945001
(Ours) Test accuracy: 0.6404177456199935
(Theirs) Train accuracy: 0.6434856262945001
(Theirs) Test accuracy: 0.6404177456199935
------------------------------
(Ours) Train accuracy: 0.5434331358870558
(Ours) Test accuracy: 0.5382767683216493
(Theirs) Train accuracy: 0.5434331358870558
(Theirs) Test accuracy: 0.5382767683216493
------------------------------
(Ours) Train accuracy: 0.5923178293494454
(Ours) Test accuracy: 0.590479631138747
(Theirs) Train accuracy: 0.5923178293494454
(Theirs) Test accuracy: 0.590479631138747
------------------------------
(Ours) Train accuracy: 0.5345675261236849
(Ours) Test accuracy: 0.5391597289478919
(Theirs) Train accuracy: 0.5345675261236849
(Theirs) Test accuracy: 0.5391597289478919
------------------------------
(Ours) Train accuracy: 0.6415624657014056
(Ours) Test accuracy: 0.6541603944410617
(Theirs) Train accuracy: 0.6415624657014056
(Theirs) Test accuracy: 0.6541603944410617
----------------------------

(True,
 0.5917296440344243,
 0.039919721088819346,
 0.590050341846102,
 0.04290305732151865,
 0.5917296440344243,
 0.039919721088819346,
 0.590050341846102,
 0.04290305732151865)

In [14]:
compare_accuracies("ERFR", train_data, train_targets, test_data, test_targets)

(Ours) Train accuracy: 1.211010772558373
(Ours) Test accuracy: 1.1804533771031367
(Theirs) Train accuracy: 1.211010772558373
(Theirs) Test accuracy: 1.1804533771031367
------------------------------
(Ours) Train accuracy: 1.2070874225010775
(Ours) Test accuracy: 1.1800898186449236
(Theirs) Train accuracy: 1.2070874225010775
(Theirs) Test accuracy: 1.1800898186449236
------------------------------


KeyboardInterrupt: 

# Verify MAB solvers are faster in speed but have no change in accuracy

# Classification

In [22]:
import time
from typing import Any, Tuple

# Classification #
# Vanilla random forest + H + GB + GBH
from data_structures.wrappers.random_forest_classifier import RandomForestClassifier as RFC
from data_structures.wrappers.histogram_random_forest_classifier import HistogramRandomForestClassifier as HRFC
from data_structures.wrappers.gradient_boosted_random_forest_classifier import GradientBoostedRandomForestClassifier as GBRFC
from data_structures.wrappers.gradient_boosted_histogram_random_forest_classifier import GradientBoostedHistogramRandomForestClassifier as GBHRFC

# Extremely random forest + GB (already histogrammed)
from data_structures.wrappers.extremely_random_forest_classifier import ExtremelyRandomForestClassifier as ERFC
from data_structures.wrappers.gradient_boosted_extremely_random_forest_classifier import GradientBoostedExtremelyRandomForestClassifier as GBERFC

# Random patches + H + GB + GBH
from data_structures.wrappers.random_patches_classifier import RandomPatchesClassifier as RPC
from data_structures.wrappers.histogram_random_patches_classifier import HistogramRandomPatchesClassifier as HRPC
from data_structures.wrappers.gradient_boosted_random_patches_classifier import GradientBoostedRandomPatchesClassifier as GBRPC
from data_structures.wrappers.histogram_random_patches_classifier import HistogramRandomPatchesClassifier as HBRPC


# Regression #
# Vanilla random forest + H + GB + GBH
from data_structures.wrappers.random_forest_regressor import RandomForestRegressor as RFR
from data_structures.wrappers.histogram_random_forest_regressor import HistogramRandomForestRegressor as HRFR
from data_structures.wrappers.gradient_boosted_random_forest_regressor import GradientBoostedRandomForestRegressor as GBRFR
from data_structures.wrappers.gradient_boosted_histogram_random_forest_regressor import GradientBoostedHistogramRandomForestRegressor as GBHRFR

# Extremely random forest + GB (already histogrammed)
from data_structures.wrappers.extremely_random_forest_regressor import ExtremelyRandomForestRegressor as ERFR
from data_structures.wrappers.gradient_boosted_extremely_random_forest_regressor import GradientBoostedExtremelyRandomForestRegressor as GBERFR

# Random patches + H + GB + GBH
from data_structures.wrappers.random_patches_regressor import RandomPatchesRegressor as RPR
from data_structures.wrappers.histogram_random_patches_regressor import HistogramRandomPatchesRegressor as HRPR
from data_structures.wrappers.gradient_boosted_random_patches_regressor import GradientBoostedRandomPatchesRegressor as GBRPR
from data_structures.wrappers.histogram_random_patches_regressor import HistogramRandomPatchesRegressor as HBRPR

In [23]:
def time_measured_fit(
    model: Any,
) -> float:
    """
    Returns wall clock time of training the model, in seconds.
    
    Has a side effect: trains the model.
    """
    start = time.time()
    model.fit()
    end = time.time()
    return end - start

In [49]:
from utils.constants import GINI, BEST, EXACT, MAB, MSE


def compare_runtimes(
    compare: str = "HRFC",
    train_data: np.ndarray = None,
    train_targets: np.ndarray = None,
    test_data: np.ndarray = None,
    test_targets: np.ndarray = None,
    num_seeds: int = 10,
) -> bool:
    # Runtimes
    our_train_times = []
    their_train_times = []
    
    # For accuracies
    our_train_accs = []
    our_test_accs = []
    their_train_accs = []
    their_test_accs = []
    for seed in range(num_seeds):
        # Ok to have n_jobs = -1 throughout?
        if compare == "HRFC":
            our_model = HRFC(data=train_data, labels=train_targets, n_estimators=5, max_depth=5, min_samples_split=2, min_impurity_decrease=0, max_leaf_nodes=None, budget=None, criterion=GINI, splitter=BEST, solver=MAB, random_state=seed, verbose=False)
            their_model = HRFC(data=train_data, labels=train_targets, n_estimators=5, max_depth=5, min_samples_split=2, min_impurity_decrease=0, max_leaf_nodes=None, budget=None, criterion=GINI, splitter=BEST, solver=EXACT, random_state=seed, verbose=False)
        else:
            raise NotImplementedError("Need to decide what models to compare")
        
        assert 'sklearn' not in their_model.__module__, "Cannot use sklearn models for runtime comparisons"
        
        our_runtime = time_measured_fit(our_model)
        our_train_times.append(our_runtime)

        their_runtime = time_measured_fit(their_model)
        their_train_times.append(their_runtime)

        if compare == "RFC" or compare == "ERFC" or compare == "HRFC":
            our_train_acc = np.mean(our_model.predict_batch(train_data)[0] == train_targets)
            our_test_acc = np.mean(our_model.predict_batch(test_data)[0] == test_targets)
            their_train_acc = np.mean(their_model.predict_batch(train_data)[0] == train_targets)
            their_test_acc = np.mean(their_model.predict_batch(test_data)[0] == test_targets)

        print("(Ours) Train accuracy:", our_train_acc)
        print("(Ours) Test accuracy:", our_test_acc)
        print("(Theirs) Train accuracy:", their_train_acc)
        print("(Theirs) Test accuracy:", their_test_acc)
        print("*" * 30)
        print("(Ours) Runtime:", our_runtime)
        print("(Theirs) Runtime:", their_runtime)
        print("-" * 30)

        our_train_accs.append(our_train_acc)
        our_test_accs.append(our_test_acc)
        their_train_accs.append(their_train_acc)
        their_test_accs.append(their_test_acc)
    
    # For accuracies
    our_avg_train = np.mean(our_train_accs)
    our_std_train = np.std(our_train_accs)

    our_avg_test = np.mean(our_test_accs)
    our_std_test = np.std(our_test_accs)
    
    their_avg_train = np.mean(their_train_accs)
    their_std_train = np.std(their_train_accs)
    
    their_avg_test = np.mean(their_test_accs)
    their_std_test = np.std(their_test_accs)
    
    # For runtimes
    our_avg_train_time = np.mean(our_train_times)
    our_std_train_time = np.std(our_train_times)
    
    their_avg_train_time = np.mean(their_train_times)
    their_std_train_time = np.std(their_train_times)
    
    
    # See if confidence intervals overlap
    overlap = np.abs(their_avg_test - our_avg_test) < their_std_test + our_std_test
    return overlap, our_avg_train, our_std_train, our_avg_test, our_std_test, their_avg_train, their_std_train, their_avg_test, their_std_test, our_avg_train_time, our_std_train_time, their_avg_train_time, their_std_train_time

In [48]:
test_labels = ng_test.target
compare_runtimes("HRFC", pca_train_vecs_huge, pca_train_labels_huge, pca_test_vecs, test_labels)

(Ours) Train accuracy: 0.8713886300093197
(Ours) Test accuracy: 0.7755960729312763
(Theirs) Train accuracy: 0.8872320596458527
(Theirs) Test accuracy: 0.7840112201963534
******************************
(Ours) Runtime: 19.933643102645874
(Theirs) Runtime: 25.426739931106567
------------------------------
(Ours) Train accuracy: 0.8732525629077353
(Ours) Test accuracy: 0.761570827489481
(Theirs) Train accuracy: 0.8536812674743709
(Theirs) Test accuracy: 0.7741935483870968
******************************
(Ours) Runtime: 23.75546097755432
(Theirs) Runtime: 25.384379148483276
------------------------------
(Ours) Train accuracy: 0.8657968313140727
(Ours) Test accuracy: 0.788218793828892
(Theirs) Train accuracy: 0.848089468779124
(Theirs) Test accuracy: 0.758765778401122
******************************
(Ours) Runtime: 23.601354837417603
(Theirs) Runtime: 24.36001205444336
------------------------------
(Ours) Train accuracy: 0.8630009319664492
(Ours) Test accuracy: 0.7489481065918654
(Theirs) Tr

(True,
 0.8575955265610439,
 0.013809466126368275,
 0.7622720897615709,
 0.016080174797061916,
 0.8689655172413794,
 0.012209827575144324,
 0.7719495091164095,
 0.019425850988141116,
 20.838464999198912,
 2.094346177385369,
 23.885877633094786,
 1.6506556262047691)

In [41]:
from mnist import MNIST

mndata = MNIST('mnist/')

train_images, train_labels = mndata.load_training()
test_images, test_labels = mndata.load_testing()

train_images = np.array(train_images)
train_labels = np.array(train_labels)
test_images = np.array(test_images)
test_labels = np.array(test_labels)


In [39]:
np.array(train_images).shape

(60000, 784)

In [50]:
compare_runtimes("HRFC", train_images, train_labels, test_images, test_labels)

/var/folders/2z/dxb5rnhd7wx6yg77m3z4c1n40000gn/T/ipykernel_44267/2479672015.py:39: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  our_test_acc = np.mean(our_model.predict_batch(test_data)[0] == test_targets)
/var/folders/2z/dxb5rnhd7wx6yg77m3z4c1n40000gn/T/ipykernel_44267/2479672015.py:41: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  their_test_acc = np.mean(their_model.predict_batch(test_data)[0] == test_targets)


(Ours) Train accuracy: 0.7571
(Ours) Test accuracy: 0.0
(Theirs) Train accuracy: 0.7843333333333333
(Theirs) Test accuracy: 0.0
******************************
(Ours) Runtime: 63.211597204208374
(Theirs) Runtime: 277.41305923461914
------------------------------
(Ours) Train accuracy: 0.75775
(Ours) Test accuracy: 0.0
(Theirs) Train accuracy: 0.7789833333333334
(Theirs) Test accuracy: 0.0
******************************
(Ours) Runtime: 64.38117504119873
(Theirs) Runtime: 257.7257170677185
------------------------------
(Ours) Train accuracy: 0.7639
(Ours) Test accuracy: 0.0
(Theirs) Train accuracy: 0.75385
(Theirs) Test accuracy: 0.0
******************************
(Ours) Runtime: 61.45366072654724
(Theirs) Runtime: 222.81906414031982
------------------------------
(Ours) Train accuracy: 0.7476666666666667
(Ours) Test accuracy: 0.0
(Theirs) Train accuracy: 0.7629666666666667
(Theirs) Test accuracy: 0.0
******************************
(Ours) Runtime: 57.80263113975525
(Theirs) Runtime: 220.

(False,
 0.7609633333333333,
 0.006162497689880117,
 0.0,
 0.0,
 0.7648366666666666,
 0.010460448790032337,
 0.0,
 0.0,
 55.980652046203616,
 5.100300028883073,
 228.00196406841278,
 20.760141606448816)

In [43]:
from sklearn.ensemble import RandomForestClassifier as RFC_sklearn
rfc = RFC_sklearn()
rfc.fit(train_images, train_labels)

RandomForestClassifier()

In [45]:
np.mean(rfc.predict(test_images) == test_labels)

0.9689